<a href="https://colab.research.google.com/github/liorhirsch/Kaggle-Landmark-ML-Challange/blob/zolantz/TrainData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Upload data from google drive

In [0]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import zipfile
with zipfile.ZipFile("drive/My Drive/content/all.zip", 'r') as zip_ref:
    zip_ref.extractall("photos")

# Build training data


In [0]:

import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from joblib import Parallel, delayed
import multiprocessing

**Upload to memory all the photos.**

The data will be array of arrays. Each sub-array represents one landmark and will contain tuples of landmark photo and landmark id.

In [0]:
DATADIR = "photos"
CATEGORIES =  os.listdir(DATADIR);
training_data = []
IMG_SIZE = 100
   
def create_training_data(category):
    data = []
    print("Building training data for landmark id %s" %category);
    path = os.path.join(DATADIR, category)  # path tolandmark photos
    landmark_id = category
    for img in os.listdir(path):
        try:
            img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
            new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
            data.append([new_array, landmark_id])
        except Exception as e:
            pass   # do nothing
    return data;
          
num_cores = multiprocessing.cpu_count();
training_data_nested=Parallel(n_jobs=num_cores)(delayed(create_training_data)(category) for category in CATEGORIES)
# training_data=[filename for path in dirs for filename in os.listdir(path)]


**FlatMap the data.**

After this proccess, there will be a list of tuples. each tuple contain image and id of single landmark.

In [0]:
training_data=[landmark_id_pair for landmark_photos in training_data_nested for landmark_id_pair in landmark_photos]

In [0]:
import random

random.shuffle(training_data)

Build the training data as expected to keras library.




In [0]:
X = []
y = []

for features, label in training_data:
    X.append(features)
    y.append(label)

# We need to convert the List of images to a numpy array for Tensorflow
# We create a numpy array where the image is in the shape of (50,50,1). 1 since this is a gray scale
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
print(X[0].shape)

(100, 100, 1)


# Train

In [0]:

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, Conv3D
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping

**Normalize the images.**

In [0]:
X = X/255.0


In [0]:
model = Sequential()
model.add(Conv2D(64, (3,3), input_shape = X.shape[1:]))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Conv2D(64, (3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

#Flatening the data
model.add(Flatten())
model.add(Dense(120))
model.add(Activation("relu"))


In [0]:
model.summary()


In [0]:
# adamOptimizer = tf.keras.optimizers.Adam(lr=0.001)

# model.compile(loss = 'sparse_categorical_crossentropy', 
#              optimizer='adam',
#              metrics=['accuracy'])

model.compile(loss='categorical_crossentropy', optimizer='adam',\
 metrics=['accuracy'])

In [0]:

# model.fit(X, y, batch_size=32, epochs = 3, validation_split=0.1)

model.fit(X, y, batch_size=64, epochs=1, verbose=1, \
validation_split=0.2, shuffle=True)

AttributeError: ignored

In [0]:
train_loss, train_acc = model.evaluate(X,y)
